Insight 6 - Which zipcode area has the lowest Price-to-Wage ratio (Aim to indentify the best places to live for new immigrants)

In [1]:
import os
from spark_utils import *
from pyspark.sql import functions as F

spark = create_spark_session()
bucket = 's3a://helenaudacitybucket'

Note: Here we take the LCA income that is based on the income of H1b holders. 
The prevailing wage data seems biased (They probably intentionally remove serveral high income data points from prevailing wage data.)

In [2]:
load_zillow = spark.read.parquet(os.path.join(bucket, 'processed_data', 'Zillow_price_rent'))
load_zillow.createOrReplaceTempView('zillow')
load_lca = spark.read.parquet(os.path.join(bucket, 'processed_data', 'LCA'))
load_lca.createOrReplaceTempView('lca')

In [3]:
output = spark.sql("""
SELECT zillow.State, 
zillow.Metro, 
zillow.CountyName, 
zillow.Zipcode, 
zillow.2021_07_Price,
(zillow.2021_07_Price/AVG_ANNUAL_INCOME) AS Price_Wage_Ratio,
AVG_ANNUAL_INCOME,
INCOME_SAMPLE_SIZE
FROM zillow
JOIN (SELECT WORKSITE_POSTAL_CODE, 
             Avg(ANNUAL_INCOME) AS AVG_ANNUAL_INCOME,
             Count(ANNUAL_INCOME) AS INCOME_SAMPLE_SIZE
      FROM lca
      GROUP BY WORKSITE_POSTAL_CODE
) lca_wage_by_zip
ON lca_wage_by_zip.WORKSITE_POSTAL_CODE = zillow.Zipcode
WHERE INCOME_SAMPLE_SIZE>10
ORDER BY Price_Wage_Ratio
LIMIT 10
""")
output.limit(100).toPandas()

,State,Metro,CountyName,Zipcode,2021_07_Price,Price_Wage_Ratio,AVG_ANNUAL_INCOME,INCOME_SAMPLE_SIZE
0,NY,New York-Newark-Jersey City,Westchester County,10601,565868.0,0.003686,1.535205e+08,30
1,NY,New York-Newark-Jersey City,New York County,10019,1257285.0,0.070547,1.782192e+07,343
2,AL,Birmingham-Hoover,Jefferson County,35205,196079.0,0.220144,8.906867e+05,13
3,FL,Lakeland-Winter Haven,Polk County,33801,187886.0,0.355401,5.286596e+05,12
4,GA,Atlanta-Sandy Springs-Roswell,Gwinnett County,30093,249234.0,0.378466,6.585369e+05,16
5,OH,Columbus,Franklin County,43202,311780.0,0.420067,7.422145e+05,12
6,NJ,New York-Newark-Jersey City,Middlesex County,8817,377523.0,0.497118,7.594233e+05,183
7,CA,Riverside-San Bernardino-Ontario,San Bernardino County,91761,586845.0,0.497725,1.179054e+06,22
8,DE,Philadelphia-Camden-Wilmington,New Castle County,19801,112191.0,0.539172,2.080803e+05,122
9,PA,Pittsburgh,Allegheny County,15203,234197.0,0.550752,4.252316e+05,23


*Take home message:* if you are looking for a job, open for relocation, and have the desire to buy a home, check the opportunies in these cities. A low Price-to-Wage indicates that the place is nice to live, affordable, and potentially attracts new immigrants therefore have an increase in value.

Based on the income and Price_Wage_Ratio in this table, it seems that some H1b immigrants might be able to buy a new home every 2 years.